<a href="https://colab.research.google.com/github/fzanart/GHDomains/blob/main/New1000popular_repos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%%capture
!pip install PyGithub

In [25]:
import pandas as pd
from google.colab import drive
from github import Github, RateLimitExceededException
from github.GithubException import UnknownObjectException
from tqdm import tqdm
from datetime import datetime
import time
import os
import json
import pickle
from getpass import getpass
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
new_1000 = pd.read_csv('/content/drive/MyDrive/GHDomains/2000Popular_repos.csv', index_col=0)
new = new_1000['Name'].to_list()

In [27]:
old_1000 = pd.read_csv('/content/drive/MyDrive/GHDomains/expanded_frame_final.csv', usecols=['Name'])
old = old_1000['Name'].to_list()

In [28]:
repos_ids = list(set(new) - set(old))

In [29]:
len(repos_ids)

893

In [30]:
with open('/content/drive/MyDrive/GHDomains/new_popular_list.pickle', 'wb') as fp:
    pickle.dump(repos_ids, fp)

In [31]:
token = getpass(prompt="Enter your Github token:")
g = Github(token, timeout=30, per_page=100)

Enter your Github token:··········


In [32]:
def catch(c):
    try:
        return c.author.login
    except:
        return c.commit.author.name


def get_commits_data(token, repo_id):

    repo = token.get_repo(repo_id)

    repo_commits_author = []
    repo_commits_message = []
    try:
        
        commits = repo.get_commits()
        repo_commits_author.extend([catch(commit) for commit in commits])
        repo_commits_message.extend([commit.commit.message for commit in commits])
        
        if len(repo_commits_author) != 0 and len(repo_commits_author) !=0:
            return {'Authors:': repo_commits_author, 'Messages:': repo_commits_message}
        else:
            return None
    except:
	    return None

def download_commits(token, repo_id):

    return get_commits_data(token, repo_id)

In [33]:
output_directory_path    = '/content/drive/MyDrive/GHDomains/Downloaded_repos'
repos_already_downloaded = [file_name.replace('.json', '').replace(':','/') for file_name in os.listdir(output_directory_path) if file_name.endswith('.json') and not file_name.startswith('all_repos')]
repos_to_download        = [repo_name for repo_name in repos_ids if repo_name not in repos_already_downloaded]

In [34]:
len(repos_to_download)

453

In [35]:
for i, repo_id in enumerate(repos_to_download):
  try:
    print('Downloading repo: ' + repo_id + ' ...')
    print(str(time.asctime(time.localtime(time.time()))) + ' ' + ': #' + str(i+1+len(repos_already_downloaded)) + ' out of ' + str(len(repos_ids)) + ' repos')

    commits = download_commits(g, repo_id)
    repo_id = repo_id.replace('/', ':')
    json.dump(commits, open(output_directory_path + '/' + repo_id + '.json', 'w'))

    print('- '*50)
  except RateLimitExceededException:
    print('sleeping...')
    time.sleep((g.get_rate_limit().core.reset - datetime.today()).seconds + 5)
    print('continue...')
  except UnknownObjectException:
    print(repo_id + ' not found')
    continue

Mon Mar 21 01:54:12 2022 : #441 out of 893 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Mon Mar 21 01:54:15 2022 : #442 out of 893 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Mon Mar 21 01:54:15 2022 : #443 out of 893 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Mon Mar 21 01:54:16 2022 : #444 out of 893 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Mon Mar 21 01:54:18 2022 : #445 out of 893 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Mon Mar 21 01:54:19 2022 : #446 out of 893 repos
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Mon Mar 21 01:54:21 2022 : #447 out of 893 repos
- - - - - - - - - - - - - - - - - - - - - - - - - -

In [45]:
commits = []
for repo_id in repos_ids:
  repo = g.get_repo(repo_id)
  commits.extend([(repo.full_name, repo.get_commits().totalCount)])

In [46]:
df = pd.DataFrame(commits, columns =['Name', 'commits'])

In [53]:
df.sort_values(by='commits', ascending=False)


,Name,commits
357,chromium/chromium,1104506
164,aosp-mirror/platform_frameworks_base,687560
95,llvm/llvm-project,418533
82,Homebrew/homebrew-core,282918
425,Homebrew/homebrew-cask,146749
...,...,...
745,soumith/ganhacks,6
0,kelseyhightower/nocode,4
52,nodejs/node-v0.x-archive,2
450,lettier/3d-game-shaders-for-beginners,1


In [97]:
# Computing IQR
df['commits'].quantile(0.25)

291.0

In [98]:
df['commits'].quantile(0.75)

2556.0

In [100]:
df['commits'].quantile(0.996)

205574.00800000224

In [103]:
Q3 + 1.5 * IQR

137344.699999999